In [1]:
import os
import subprocess
import numpy as np
import pandas as pd

In [2]:
def find_files(filename, search_path):
   result = []
   # Walking top-down from the root
   for root, dir, files in os.walk(search_path):
      if filename in files:
         result.append(os.path.join(root, filename))
   return result

def get_path(str):
   str = str.replace('<', '').replace('>', '').replace('"', '')
   return str.split('/')[::-1]

def resolve_include(possible_files, ref_path):
   ref_pos = 1
   if len(possible_files) == 0:
      return '(unresolved)'
   else:
      # no need to check if there is nothing to check
      if len(ref_path) == 1:
         return possible_files[0]
      else:
         # check if matches
         for file in possible_files:
            if ref_path[ref_pos] + "/" in file:
               return file
         # if not exact match, return first
         # todo: go back other folders
         return possible_files[0]
      
def get_unique_file_names(includes):
   arr = []
   for item in includes:
      path = get_path(item)
      arr.append(path[0])
   arr = np.unique(np.array(arr))
   arr = filter(lambda item: ("." in item), arr)
   arr = list(arr)
   arr = list(map(lambda item: (".*/" + item), arr))
   return arr   

In [3]:
arr_res_2nd = []
arr_unresolved = []
engine_path = '/home/ullmann/Documents/research/engine_repos_local/urho3d'

# 1 - load
ds = pd.read_csv('errors.txt', names=['file', 'includes'], header=None)[0:1000]
ds = ds['includes'].values

# 2 - build command string
unique_filenames = get_unique_file_names(ds)
# todo: concatenate the dot slash before
str_command = "find . -type f -regex '" + ("\|".join(unique_filenames)) + "'"
str_command = str_command + " > bobo.txt"

# 3 - execute and save
# receive this path by parameter
os.chdir('/home/ullmann/Documents/research/engine_repos_local/urho3d')
os.system(str_command)
result = list(open('bobo.txt', 'r'))

# 4 - resolve one by one
for filename in unique_filenames:
    xinaveia = filename.replace(".*/", "")
    xups = list(filter(lambda item : xinaveia in item, result))
    print(xups, xinaveia)
    if len(xups) > 0:
        ref_path = get_path(xups[0])
        possible_files = list(filter(lambda item : xinaveia in item, result))
        risolvido = resolve_include(possible_files, ref_path)
        arr_res_2nd.append(risolvido)
    else:
        arr_unresolved.append(filename)

['./Source/Urho3D/Core/Context.h\n'] Context.h
['./Source/ThirdParty/Assimp/include/assimp/DefaultIOStream.h\n'] DefaultIOStream.h
['./Source/ThirdParty/Assimp/include/assimp/DefaultIOSystem.h\n'] DefaultIOSystem.h
['./Source/ThirdParty/Assimp/include/assimp/DefaultLogger.hpp\n'] DefaultLogger.hpp
['./Source/ThirdParty/Assimp/include/assimp/Defines.h\n'] Defines.h
['./Source/ThirdParty/Assimp/include/assimp/Exporter.hpp\n'] Exporter.hpp
['./Source/Urho3D/IO/FileSystem.h\n'] FileSystem.h
['./Source/ThirdParty/Assimp/include/assimp/IOStream.hpp\n'] IOStream.hpp
['./Source/ThirdParty/Assimp/include/assimp/IOSystem.hpp\n'] IOSystem.hpp
['./Source/ThirdParty/Assimp/include/assimp/Importer.hpp\n'] Importer.hpp
['./Source/ThirdParty/Assimp/include/assimp/LogStream.hpp\n'] LogStream.hpp
['./Source/ThirdParty/Assimp/include/assimp/NullLogger.hpp\n'] NullLogger.hpp
[] OSAtomic.h
['./Source/ThirdParty/Assimp/contrib/openddlparser/include/openddlparser/OpenDDLParser.h\n'] OpenDDLParser.h
['./Sourc

In [4]:
print("resolved on 2nd pass", len(arr_res_2nd))
print("unresolved", len(arr_unresolved))

resolved on 2nd pass 48
unresolved 14


In [6]:
arr_unresolved

['.*/OSAtomic.h',
 '.*/c4d_file.h',
 '.*/default_alien_overloads.h',
 '.*/direct.h',
 '.*/mmsystem.h',
 '.*/param.h',
 '.*/ppu_asm_intrinsics.h',
 '.*/regdef.h',
 '.*/revision.h',
 '.*/stat.h',
 '.*/stdarg.h',
 '.*/stddef.h',
 '.*/windows.h',
 '.*/xtl.h']